### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/brittneycontreras/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221109_scenario_projections.ipynb")

2022/10/28 10:10:06 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/10/28 10:10:12 INFO|-------------------------|{} Retrieving vaccinations data
2022/10/28 10:10:19 INFO|-------------------------|{} Constructing vaccination projections
2022/10/28 10:10:23 INFO|-------------------------|{} Retrieving hospitalizations data
2022/10/28 10:10:30 INFO|----------------------|{} Prepping Model
2022/10/28 10:10:30 INFO|-----------------------|{} Prepping Model
2022/10/28 10:12:42 INFO|----------------------|{} Model prepped for fitting in 132.11734254099997 seconds.
2022/10/28 10:12:42 INFO|----------------------|{} Will fit 25 times
2022/10/28 10:21:09 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 507.153976625 seconds: {0: {'co': 0.8306366573462861}, 14: {'co': 0.6964605500728897}, 28: {'co': 0.5712212382169275}, 42: {'co': 0.0034172328618746997}, 56: {'co': 0.6814327403468164}, 70: {'co': 0.8913015126022406}}
2022/10/28 10:21:09 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/10/28 

2022/10/28 10:48:17 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 64.091160167 seconds: {504: {'co': 0.7862507288678809}, 518: {'co': 0.7800460084633297}, 532: {'co': 0.7569254451330324}, 546: {'co': 0.7093452279970082}, 560: {'co': 0.7431796813074834}, 574: {'co': 0.7376454710769529}}
2022/10/28 10:48:17 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/10/28 10:49:50 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 89.85824170900014 seconds: {532: {'co': 0.7561941836540607}, 546: {'co': 0.711415319176938}, 560: {'co': 0.7394194790365072}, 574: {'co': 0.7493923580669691}, 588: {'co': 0.7863036945990122}, 602: {'co': 0.7652705733983328}}
2022/10/28 10:49:50 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/10/28 10:51:08 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 76.06712670900015 seconds: {560: {'

### Create and run scenarios from Feb 2022 to present

In [5]:
multiprocess = 4

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 5]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
               "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-30": vx_seed, "2022-10-30": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                            "attrs": {'age': '0-19', 'vacc': 'shot1'},
                            "mults": {"2020-01-01": 1,
                                      "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                      "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                      "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                      "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                      "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                      "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                      "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                      "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                      "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                            "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                    'hosp_reporting_frac': hrf,
                                    'tags': {'vx_seed': vx_seed,
                                             'vir_mult': vir_mult,
                                                }})
            

In [6]:
# check how many scenarios there are
len(scenario_model_args)

4

In [7]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/10/28 11:37:57 INFO|============================================================
2022/10/28 11:37:57 INFO|============================================================
2022/10/28 11:37:57 INFO|============================================================
2022/10/28 11:37:57 INFO|============================================================
2022/10/28 11:38:01 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/10/28 11:38:01 INFO|-----------|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/10/28 11:38:01 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/10/28 11:38:01 INFO|-----------|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/10/28 11:38:03 INFO|-----------|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0} Constructing vaccination projections
2022/10/28 11:38:04 INFO|-----------|{'vx_seed': 0, 'vi

2022/10/28 11:47:04 INFO|--------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/6 completed in 106.57052379200002 seconds: {823: {'co': 0.8475073365860945}, 837: {'co': 0.8455444268507887}, 851: {'co': 0.8320769226842218}, 865: {'co': 0.8611423306087927}, 879: {'co': 0.8526907443707877}, 893: {'co': 0.857555973380959}}
2022/10/28 11:47:04 INFO|---------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Running forward sim
2022/10/28 11:47:05 INFO|--------|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/6 completed in 106.58179362499999 seconds: {823: {'co': 0.8475073365860945}, 837: {'co': 0.8455444268507887}, 851: {'co': 0.8320769226842218}, 865: {'co': 0.8611423306087927}, 879: {'co': 0.8526907443707877}, 893: {'co': 0.857555973380959}}
2022/10/28 11:47:05 INFO|---------|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Running forward si

2022/10/28 11:52:39 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '5'}: Transmission control fit 6/6 completed in 126.38073574999999 seconds: {907: {'co': 0.8602518508354488}, 921: {'co': 0.8639327923914805}, 935: {'co': 0.8440621481149968}, 949: {'co': 0.858065323887688}, 963: {'co': 0.8359030939647235}, 977: {'co': 0.833367457619039}}
2022/10/28 11:52:39 INFO|---------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '5'}: Running forward sim
2022/10/28 11:52:42 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '5', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366573462861}, 14: {'co': 0.6964605500728897}, 28: {'co': 0.5671378805699239}, 42: {'co': 1.9489201927168757e-15}, 56: {'co': 0.6966643050709015}, 70: {'co': 0.833269382748188}, 84: {'co': 0.8296267084735902}, 98: {'co': 0.8316912752154394}, 112: {'co': 0.8774904787370315}, 126: {'co': 0.8243037775983401}, 140: {'co': 0.760453059907473}, 154: {'co'

2022/10/28 11:53:22 INFO|-----------------------|spec_ids: 4191,4192,4193,4194


### Run the report for each fit model

In [8]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/10/28 11:59:47 INFO|============================================================
2022/10/28 11:59:48 INFO|============================================================
2022/10/28 11:59:48 INFO|--------|Solving model
2022/10/28 11:59:48 INFO|--------|Solving model
2022/10/28 11:59:48 INFO|============================================================
2022/10/28 11:59:48 INFO|============================================================
2022/10/28 11:59:48 INFO|--------|Solving model
2022/10/28 11:59:48 INFO|--------|Solving model
2022/10/28 12:00:21 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/10/28 12:00:21 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/10/28 12:00:21 WARNING|No artists with labels found to put in legend.  Note that artists 

In [9]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/10/28 12:06:14 INFO|Projecting
2022/10/28 12:06:14 INFO|
2022/10/28 12:07:13 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '5', 'run_type': 'fit'}: Running forward sim
2022/10/28 12:07:15 INFO|
2022/10/28 12:08:15 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '5', 'run_type': 'fit'}: Running forward sim
2022/10/28 12:08:17 INFO|
2022/10/28 12:09:16 INFO|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '5', 'run_type': 'fit'}: Running forward sim
2022/10/28 12:09:18 INFO|
2022/10/28 12:10:16 INFO|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '5', 'run_type': 'fit'}: Running forward sim
2022/10/28 12:10:19 INFO|Running reports
